<div class="alert alert-block alert-success"> 
    
# How to use this notebook
    
This notebook can perform all the steps required for data analysis of SELEX-seq data with the SELEX package. To use it, follow these steps: 

### 1. Input variables
All file paths, file names and as well as information such as the variable length need to be **given as input variables** in the section indicated below. At the very end of this notebook there is a blue box giving detailed information on how to download and save data sets from the Gene Expression Omnibus, how to interpret the FASTQ files and how to save time by saving the metadata in an annotation file.
    
### 2. Setting up SELEX
The next steps require **no input - just run the cells**. Appropriate settings are set and the data is imported properly such that the SELEX package 'knows' how to deal with and address the data you have provided. 
    
### 3. Analysis
This step requires one input - indicate the round that is to be compared to R0 for the calculation of the information gain and affinities.
    
</div>

<div class="alert alert-block alert-warning"> 

In the following, all sections requiring input are highlighted in yellow.
    
</div>

<div class="alert alert-block alert-warning">

# 1. Input Variables
    
### Protein Name & Round names & Sample Name
Indicate the name of the protein being analyzed, as a string. We will use this string to name samples and output data. Likewise, indicate the name of the sample sequences the protein is analyzed with.

</div>

In [25]:
workdir="C:/Users/jmuinoa/Downloads/test" # working directory
pname = 'FUL'               # name of the TF/protein to be analyzed

#### <div class="alert alert-block alert-warning">

### File paths
    

The program neeed the filepath for the fasta/fastq files containing the seqeuncing of the DNA oligos for round 0 (r0_filepath) and one round of enrichment (r_filepath). The parameter round_en indicates which round is used in r_filepath (eg: round 4 of enrichment)
    </div>

In [26]:
r0_filepath = 'data/FUL-FUL_40N_Round0_test.fastq.gz'
r_filepath = 'data/FUL-FUL_40N_Round0_test.fastq.gz'
round_en = 4 #integer

<div class="alert alert-block alert-warning">

### Variable length, Left and Right Barcodes
This is the length of the variable region of the nucleic acid snippets in the sample. If sequences are purely random with no defined left or right barcodes, the length of the variable region is equal to the length of the sequence.
    
</div>

In [27]:
variable_length = 40
left_bc = ''
right_bc = ''

# 2.a) Required settings for SELEX

In [28]:
setwd(workdir)
suppressMessages(library(SELEX))
options(java.parameters="-Xmx2500M") # Set maximum memory usage limit

The working directory contains the temporary files. The folder is created if it does not exist, everything in it is automatically kept track of and does not need to be read or addressed by a person. Once calculation is complete, the contents or even the whole folder can safely be deleted. 

In [29]:
tempDir = "./cache/" # Specify the temporal working directory
selex.config(workingDir=tempDir, maxThreadNumber=4) # Configure Selex with 4 maxThreadNumber

Reading the setting file:C:\Users\jmuinoa\Documents/SELEX.properties
Saving the setting file:C:\Users\jmuinoa\Documents/SELEX.properties


If rJava is propery initialized and installed, the method jvmStatus should show you the current memory usage of the Java Virtual Machine. You can check this if you run into any trouble.

In [8]:
#selex.jvmStatus()

# 2.b) Import your own data

Detailed instructions for downloading data that is available on the GEO are given below, in the last cell in this notebook.

#### Define metadata

This step is required to save metadata such as the filepath, barcodes and length of the variable region. This information is necessary for SELEX to interpret the data in subsequent steps. 

In [16]:
rname=paste("R",round_en,sep="")
sample_r0 = selex.defineSample(seqName = paste(pname, 'R0', sep='-'), seqFile = r0_filepath, 
                               sampleName = paste('R0', pname, sep='.'), round = 0, varLength = variable_length, 
                               leftBarcode = left_bc, rightBarcode = right_bc)

sample_r = selex.defineSample(seqName = paste(pname, rname, sep='-'), seqFile = r_filepath, 
                               sampleName = paste(rname, pname, sep='.'), round = round_en, varLength = variable_length, 
                               leftBarcode = left_bc, rightBarcode = right_bc)

#### Save metadata (i.e. annotation) for later

Print the name to check formatting. (Folders need to be separated by slashes (/) and suffixes need to be preceded by points.)

In [20]:
export_name = paste(paste(workdir,"/", pname, sep=''), 'xml', sep='.')
selex.saveAnnotation(export_name)

[1] "C:/Users/jmuinoa/Downloads/test/FUL.xml"

ERROR: Error in .jcall("RJavaTools", "Ljava/lang/Object;", "invokeMethod", cl, : java.lang.NoClassDefFoundError: javax/xml/bind/JAXBContext


#### Import the samples

In [13]:
r0 = selex.sample(seqName = paste(pname, 'R0', sep='-'), sampleName = paste('R0', pname, sep='.'), round = 0)
Round2BAnalyzed = selex.sample(seqName = paste(pname, rname, sep='-'), sampleName = paste(rname, pname, sep='.'), round = round_en)

#### Split the first sample into a training and a testing part

Default ratio is 0.5 (i.e. half training and half testing data). 

In [15]:
r0.split = selex.split(r0, ratio=c(0.5,0.5))

#### View imported data

In [18]:
selex.sampleSummary()

,seqName,sampleName,rounds,leftBarcode,rightBarcode,leftFlank,rightFlank,seqFile
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,SEP3-AGe1C1-R0,R0.RandomSequences,0,,,,,/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1-R0.fastq
6,SEP3-AGe1C1-R0.split.1,R0.RandomSequences.split.1,0,,,,,/home/shk/Schreibtisch/shk_jose/./cache//SEP3-AGe1C1-R0.R0.RandomSequences.0_split//0.part
5,SEP3-AGe1C1-R0.split.2,R0.RandomSequences.split.2,0,,,,,/home/shk/Schreibtisch/shk_jose/./cache//SEP3-AGe1C1-R0.R0.RandomSequences.0_split//1.part
4,SEP3-AGe1C1-R1,R1.RandomSequences,1,,,,,/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1-R1.fastq
2,SEP3-AGe1C1-R2,R2.RandomSequences,2,,,,,/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1-R2.fastq
3,SEP3-AGe1C1-R3,R3.RandomSequences,3,,,,,/home/shk/Schreibtisch/shk_jose/data/SEP3-AGe1C1-R3.fastq


# 3. Analysis



## Markov model

Even though random sequences of DNA are used, the frequency with which they occur is not completely free of bias - i.e. not all random sequences are equally likely to be found.

The SELEX method uses a Markov model to predict the number of occurences of each possible k-mer in round zero (R0).

### 3.1 Count $k_{max}$

Find the maximal nucleotide length $k_{max}$ such that all sequences of that length $k_{max}$ appear at least $n$ times in the data, where $n$ is the threshold value (default: 100). 

The model will be fitted to sequences of length $k_{max}$, all other sequences will be neglected for the purpose of fitting the Markov Model. Therefore, it is important that $n$ be large enough to allow for a statistically sound fit. 

In [20]:
# Find the longest length k such that all possible k-mers are found at least 100 times
kmax.value = selex.kmax(sample=r0.split$test, threshold=100, seqfilter=NULL)

Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 1 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 2 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 3 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 4 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 5 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 6 ]
Counting [SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0][ K = 7 ]
[ sample id : SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,  filter:  variableRegionIncludeRegex:null,variableRegionExcludeRegex:null,variableRegionGroupRegex:null ]
[ SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0.kmax = 6 ]


In [21]:
#Kmax.value is the value of the adviced Kmer length
kmax.value

[1] 6

### 3.2 Create and store the Markov model

Next, training and testing data as well as $k_{max}$ is passed to SELEX which creates and saves a Markov model. If the order is not specified, it will test Markov models of increasing order, selecting the optimal order.

The order of the Markov model refers to the number of nucleotides considered when inferring the conditional occurence probabilities of subsequent nucleotides, e.g. P(A|GATTC) for a Markov model of the fifth order.

In [22]:
# Requires samples to be used for training and testing, as well as the k_max value.
mm = selex.mm(sample=r0.split$train, order=NA, crossValidationSample=r0.split$test, Kmax=kmax.value)

Overwriting Kmax = 6
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 1 ]
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 2 ]
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 3 ]
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 4 ]
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 5 ]
Counting [SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0][ K = 6 ]
[ markovLength = 5 ]
[ maxR = 0.997453 ]
[ Model = MarkovModelInfo [markovLength=5, markovLengthTotalCount=5914400, markovR2=0.9974534255596722, markovCountsPath=/home/shk/Schreibtisch/shk_jose/./cache//SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.5.dat_A7FE7F4E2E78A43F892C7F3227FFA520, markovObjPath=/home/shk/Schreibtisch/shk_jose/./cache//SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.5.dat_A7FE7F4E2E78A43F892C7F3227FFA520.prob.obj2, sample=config.ExperimentReference@455b6df1, markovModelMethod=DIVISION, crossValidationSample=c

In [23]:
selex.mmSummary()

Sample,Order,MarkovModelType,R,CVSample,CVLength
<chr>,<dbl>,<chr>,<dbl>,<chr>,<int>
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,0,DIVISION,0.9698371,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,1,DIVISION,0.9886711,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,2,DIVISION,0.9947389,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,3,DIVISION,0.9966505,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,4,DIVISION,0.9974534,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6
SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0,5,DIVISION,0.9971954,SEP3-AGe1C1-R0.split.2.R0.RandomSequences.split.2.0,6


### 3.3 Information gain and optimal length

The last line printed is the maximal information gain. 

<div class="alert alert-block alert-danger">
    
If the following line throws **this error**: 
    
>Error in .jcall("RJavaTools", "Ljava/lang/Object;", "invokeMethod", cl, : java.lang.OutOfMemoryError: Java heap space
    
It helps to delete the whole cache (working directory) and then restart the notebook (using the >> Button in the menu). 

</div>

In [24]:
selex.infogain(Round2BAnalyzed, markovModel=mm)

Counting [InfoGain][ K = 5 ]
Counting [InfoGain][ K = 6 ]
Counting [InfoGain][ K = 7 ]
Counting [InfoGain][ K = 8 ]
Counting [InfoGain][ K = 9 ]
Counting [InfoGain][ K = 10 ]
Counting [InfoGain][ K = 11 ]
Counting [InfoGain][ K = 12 ]
Counting [InfoGain][ K = 13 ]
Counting [InfoGain][ K = 14 ]
Counting [InfoGain][ K = 15 ]
Counting [InfoGain][ K = 16 ]


[1] 1.526408

Check which value of k corresponds to the maximal information gain: 

In [25]:
selex.infogainSummary()

Sample,K,InformationGain,MarkovModel,MarkovModelType
<chr>,<int>,<dbl>,<chr>,<chr>
SEP3-AGe1C1-R2.R2.RandomSequences.2,5,0.3477520,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,6,0.5562881,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,7,0.8288981,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,8,1.1434254,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,9,1.4183313,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,10,1.5264085,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,11,1.3981736,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,12,1.0429063,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION
SEP3-AGe1C1-R2.R2.RandomSequences.2,13,0.5486344,SEP3-AGe1C1-R0.split.1.R0.RandomSequences.split.1.0.Order4,DIVISION


<div class="alert alert-block alert-warning">

The **optimal length** is the value of k for which the information gain is highest. Save that information in the variable below:
    
</div>

In [26]:
optimalLength = 10

Count how many times k-mers of optimal length appear: 

In [28]:
table = selex.counts(sample=Round2BAnalyzed, k=optimalLength, markovModel=mm,minCount=10) 
head(table)

,Kmer,ObservedCount,Probability,ExpectedCount
,<chr>,<int>,<dbl>,<dbl>
1,AAATAAGGAA,11740,5.313668e-06,71.39581
2,AAAAGAGGAA,10422,5.470747e-06,73.50637
3,AAAAAAGGAA,9388,8.423538e-06,113.18082
4,CAAATAAGGA,9142,3.327236e-06,44.70560
5,ATTTAAGGAA,8534,2.286702e-06,30.72471
6,CCAAATAAGG,7988,2.426840e-06,32.60764


### Calculating affinity and error

Affinities are calculated for specific rounds, so the sample needs to be specified appropriately (i.e. enter r2 for round 2 or r3 for round 3, etc.)

In [34]:
aff = selex.affinities(sample=Round2BAnalyzed, k=optimalLength, markovModel=mm,minCount=10) 

Counting [SEP3-AGe1C1-R2.R2.RandomSequences.2][ K = 10 ]
[ Lowest Count =  1 ]


In [35]:
head(aff)

,Kmer,ObservedCount,Probability,ExpectedCount,Affinity,SE
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,AAATAAGGAA,11740,5.313668e-06,71.39581,0.4965242,0.006480692
2,AAAAGAGGAA,10422,5.470747e-06,73.50637,0.4610583,0.006386976
3,AAAAAAGGAA,9388,8.423538e-06,113.18082,0.3526492,0.005147204
4,CAAATAAGGA,9142,3.327236e-06,44.70560,0.5537101,0.008189867
5,ATTTAAGGAA,8534,2.286702e-06,30.72471,0.6453201,0.009879020
6,CCAAATAAGG,7988,2.426840e-06,32.60764,0.6060411,0.009589547


### Save and export the affinities table

In [45]:
export_table_name = paste(paste('', pname, sep=''), RoundName, 'affinities.csv', sep='_')

export_table_name

[1] "data/SEP3-AGe1C1_R2_affinities.csv"

In [46]:
write.csv(aff, export_table_name)

# Resources

<div class="alert alert-block alert-info"><b>Credit:</b> This notebook is adapted from the SELEX R package described in [Paper describing the SELEX-seq anaysis method][selex paper]: Cofactor binding evokes latent differences in DNA binding specificity between Hox proteins, Slattery et al., 2011.
[SELEX R package Manual][selex manual]</div>





[selex example]: https://bioconductor.org/packages/release/bioc/vignettes/SELEX/inst/doc/SELEX.pdf
[selex manual]: https://bioconductor.org/packages/release/bioc/manuals/SELEX/man/SELEX.pdf
[selex paper]: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3319069/